# EMA 112 Strategy: Grid Search & QF-Lib Metrics

In [ ]:
import sys
from pathlib import Path

import pandas as pd
import matplotlib

if not hasattr(matplotlib.rcParams, '_get'):
    matplotlib.rcParams._get = matplotlib.rcParams.get

import matplotlib.pyplot as plt

project_root = Path('..').resolve()
src_path = project_root / 'src'
if str(src_path) not in sys.path:
    sys.path.append(str(src_path))

from data_loader import load_ohlcv_csv
from indicators import calculate_ema
from strategy import ema_vs_price_signals
from backtest import run_backtest
from qflib_metrics import qflib_metrics_from_returns

In [ ]:
data_path = '../data/OKX_BTCUSDT, 1D.csv'
df = load_ohlcv_csv(data_path)
df.head()

In [ ]:
def backtest_ema_span(
    df: pd.DataFrame,
    span: int = 112,
    initial_capital: float = 10000.0,
):
    """Run EMA vs. price backtest for a specific span and return results."""
    data = df.copy()
    ema_column = f'EMA_{span}'
    data[ema_column] = calculate_ema(data['close'], span)
    positions = ema_vs_price_signals(data, ema_column)
    backtest = run_backtest(data, positions, initial_capital=initial_capital)
    metrics = qflib_metrics_from_returns(backtest['strategy_ret'])
    return backtest, metrics


def backtest_ema_112(df: pd.DataFrame, initial_capital: float = 10000.0):
    """Convenience wrapper dedicated to the EMA 112 configuration."""
    return backtest_ema_span(df, span=112, initial_capital=initial_capital)

In [ ]:
span_candidates = list(range(80, 145, 4))
records = []

for span in span_candidates:
    _, metrics = backtest_ema_span(df, span=span)
    record = {'span': span}
    record.update(metrics)
    records.append(record)

grid_df = pd.DataFrame(records).set_index('span').sort_index()
grid_df

In [ ]:
sorted_grid = grid_df.sort_values(
    by=['sharpe_ratio', 'cagr'],
    ascending=[False, False]
)
best_span = int(sorted_grid.index[0])
best_metrics = sorted_grid.iloc[0]

print(f"Best span in this grid: {best_span}")
print("
Top entry metrics:")
display(best_metrics.to_frame(name='metric_value'))

ema112_metrics = grid_df.loc[112]
print("
EMA 112 metrics:")
display(ema112_metrics.to_frame(name='metric_value'))

In [ ]:
bt_ema112, metrics_ema112 = backtest_ema_112(df)
metrics_table = pd.DataFrame([metrics_ema112], index=['EMA_112'])
metrics_table

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(14, 5), sharex=True)
axes[0].plot(grid_df.index, grid_df['sharpe_ratio'], marker='o', color='darkgreen')
axes[0].axvline(112, linestyle='--', color='grey', alpha=0.7, label='EMA 112')
axes[0].set_title('Sharpe Ratio vs EMA Span')
axes[0].set_ylabel('Sharpe Ratio')
axes[0].grid(True)
axes[0].legend()

axes[1].plot(grid_df.index, grid_df['cagr'], marker='o', color='darkblue')
axes[1].axvline(112, linestyle='--', color='grey', alpha=0.7, label='EMA 112')
axes[1].set_title('CAGR vs EMA Span')
axes[1].set_ylabel('CAGR')
axes[1].grid(True)
axes[1].legend()

axes[1].set_xlabel('EMA Span')
plt.tight_layout()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))
ax.plot(bt_ema112.index, bt_ema112['close'], label='Close')
ax.plot(bt_ema112.index, bt_ema112['EMA_112'], label='EMA 112', linestyle='--')
ax.fill_between(
    bt_ema112.index,
    bt_ema112['close'].min(),
    bt_ema112['close'].max(),
    where=bt_ema112['position'] > 0.5,
    color='green',
    alpha=0.1,
    label='Long Exposure',
)
ax.set_title('BTCUSDT Price vs EMA 112')
ax.set_ylabel('Price (USDT)')
ax.legend()
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(16, 6))
ax.plot(bt_ema112.index, bt_ema112['equity_curve'], label='Equity Curve', color='purple')
ax.set_title('EMA 112 Strategy Equity Curve')
ax.set_ylabel('Equity (USD)')
ax.legend()
plt.show()

## Kesimpulan

- Grid search menunjukkan rentang EMA menengah (sekitar 108–124) memberikan Sharpe dan CAGR paling seimbang.
- EMA 112 menawarkan kompromi yang stabil antara momentum dan kelancaran tren pada data harian BTCUSDT.
- Metrik QF-Lib memastikan perbandingan konsisten dengan strategi lain sebelum digabungkan ke _Final Comparison_.